In [ ]:
import pandas as pd

game_log = pd.read_csv('game_log.csv')
park_codes = pd.read_csv('park_codes.csv')
person_codes = pd.read_csv('person_codes.csv')
team_codes = pd.read_csv('team_codes.csv')
appearance_type = pd.read_csv('appearance_type.csv')
print(game_log.head())
print(park_codes.head())
print(person_codes.head())
print(team_codes.head())

/dataquest/system/env/python3/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (12,13,14,15,19,20,81,82,83,84,85,86,87,88,93,94,95,96,97,98,99,100,105,106,108,109,111,112,114,115,117,118,120,121,123,124,126,127,129,130,132,133,135,136,138,139,141,142,144,145,147,148,150,151,153,154,156,157,160) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


       date  number_of_game day_of_week v_name v_league  v_game_number h_name  \
0  18710504               0         Thu    CL1      NaN              1    FW1   
1  18710505               0         Fri    BS1      NaN              1    WS3   
2  18710506               0         Sat    CL1      NaN              2    RC1   
3  18710508               0         Mon    CL1      NaN              3    CH1   
4  18710509               0         Tue    BS1      NaN              2    TRO   

  h_league  h_game_number  v_score       ...         h_player_7_name  \
0      NaN              1        0       ...              Ed Mincher   
1      NaN              1       20       ...            Asa Brainard   
2      NaN              1       12       ...              Pony Sager   
3      NaN              1       12       ...                Ed Duffy   
4      NaN              1        9       ...            Steve Bellan   

   h_player_7_def_pos h_player_8_id  h_player_8_name h_player_8_def_pos  \
0    

In [ ]:
import sqlite3

def run_query(q):
    with sqlite3.connect('mlb.db') as conn:
        return pd.read_sql(q, conn)

def run_command(q):
    with sqlite3.connect('mlb.db') as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(q)

with sqlite3.connect('mlb.db') as conn:
    try:
        game_log.to_sql('game_log', conn, index=False)
    except Exception:
        pass
    try:
        park_codes.to_sql('park_codes', conn, index=False)
    except Exception:
        pass
    try:
        person_codes.to_sql('person_codes', conn, index=False)
    except Exception:
        pass
    try:
        team_codes.to_sql('team_codes', conn, index=False)
    except Exception:
        pass
    try:
        appearance_type.to_sql('appearance_type_table', conn, index=False)
    except Exception:
        pass

print(run_query('SELECT * FROM sqlite_master WHERE TYPE="table";'))

try:
    run_command('ALTER TABLE game_log ADD COLUMN game_id TEXT;')
    run_command('UPDATE game_log SET game_id = h_name || date || number_of_game;')
except Exception:
    pass

print(run_query('SELECT game_id FROM game_log LIMIT 5;'))

This is the markdown cell describing opportunities to normalize the data that have been observed.

![Database Schema](database-schema.png)

In [ ]:
person_exec = """CREATE TABLE person(
                     person_id TEXT PRIMARY KEY,
                     first_name TEXT,
                     last_name TEXT);"""
person_fill = """INSERT INTO person
                 SELECT id, first, last FROM person_codes;"""
try:
    run_command(person_exec)
    run_command(person_fill)
except Exception:
    pass
person_check = """SELECT * FROM person LIMIT 5;"""
print(run_query(person_check))

print()
print()
print()

park_exec = """CREATE TABLE park(
                   park_id TEXT PRIMARY KEY,
                   name TEXT,
                   nickname TEXT,
                   city TEXT,
                   state TEXT,
                   notes TEXT);"""
park_fill = """INSERT INTO park
               SELECT park_id, name, aka, city, state, notes FROM park_codes;"""
try:
    run_command(park_exec)
    run_command(park_fill)
except Exception:
    pass
park_check = """SELECT * FROM park LIMIT 5;"""
print(run_query(park_check))

print()
print()
print()

league_exec = """CREATE TABLE league(
                    league_id TEXT PRIMARY KEY,
                    name TEXT);"""
league_fill = """INSERT INTO league VALUES
                     ("NL", "National League"),
                     ("AL", "American League"),
                     ("AA", "American Association"),
                     ("FL", "Federal League"),
                     ("PL", "Players League"),
                     ("UA", "Union Association");"""
try:
    run_command(league_exec)
    run_command(league_fill)
except Exception:
    pass
league_check = """SELECT * FROM league;"""
print(run_query(league_check))

print()
print()
print()

appearance_exec = """CREATE TABLE appearance_type(
                         appearance_type_id TEXT PRIMARY KEY,
                         name TEXT,
                         category TEXT);"""
appearance_fill = """INSERT INTO appearance_type
                     SELECT appearance_type_id, name, category
                     FROM appearance_type_table;"""
try:
    run_command(appearance_exec)
    run_command(appearance_fill)
except Exception:
    pass
appearance_check = """SELECT * FROM appearance_type LIMIT 5;"""
print(run_query(appearance_check))

In [ ]:
team_exec = """CREATE TABLE team(
                   team_id TEXT PRIMARY KEY,
                   league_id TEXT,
                   city TEXT,
                   nickname TEXT,
                   franch_id TEXT,
                   FOREIGN KEY (league_id) REFERENCES league(league_id));"""
team_fill = """INSERT INTO team
               SELECT team_id, league, city, nickname, franch_id FROM team_codes
               GROUP BY 1;"""
try:
    run_command(team_exec)
    run_command(team_fill)
except Exception:
    pass
team_check = """SELECT * FROM team LIMIT 5;"""
print(run_query(team_check))

print()
print()
print()

game_exec = """CREATE TABLE game(
                   game_id TEXT PRIMARY KEY,
                   date TEXT,
                   number_of_game INTEGER,
                   park_id TEXT,
                   length_outs INTEGER,
                   day BOOLEAN,
                   completion TEXT,
                   forfeit TEXT,
                   protest TEXT,
                   attendance INTEGER,
                   length_minutes INTEGER,
                   additional_info TEXT,
                   acquisition_info TEXT,
                   FOREIGN KEY (park_id) REFERENCES park(park_id));"""
game_fill = """INSERT INTO game
               SELECT
                   game_id,
                   date,
                   number_of_game,
                   park_id,
                   length_outs,
                   CASE
                       WHEN day_night = "D" THEN 1
                       WHEN day_night = "N" THEN 0
                       ELSE NULL
                       END
                       AS day,
                   completion,
                   forefeit,
                   protest,
                   attendance,
                   length_minutes,
                   additional_info,
                   acquisition_info
               FROM game_log;"""
try:
    run_command(game_exec)
    run_command(game_fill)
except Exception:
    pass
game_check = """SELECT * FROM game LIMIT 5;"""
print(run_query(game_check))

In [ ]:
team_exec = """CREATE TABLE team_appearance (
                   team_id TEXT,
                   game_id TEXT,
                   home BOOLEAN,
                   league_id TEXT,
                   score INTEGER,
                   line_score TEXT,
                   at_bats INTEGER,
                   hits INTEGER,
                   doubles INTEGER,
                   triples INTEGER,
                   homeruns INTEGER,
                   rbi INTEGER,
                   sacrifice_hits INTEGER,
                   sacrifice_flies INTEGER,
                   hit_by_pitch INTEGER,
                   walks INTEGER,
                   intentional_walks INTEGER,
                   strikeouts INTEGER,
                   stolen_bases INTEGER,
                   caught_stealing INTEGER,
                   grounded_into_double INTEGER,
                   first_catcher_interference INTEGER,
                   left_on_base INTEGER,
                   pitchers_used INTEGER,
                   individual_earned_runs INTEGER,
                   team_earned_runs INTEGER,
                   wild_pitches INTEGER,
                   balks INTEGER,
                   putouts INTEGER,
                   assists INTEGER,
                   errors INTEGER,
                   passed_balls INTEGER,
                   double_plays INTEGER,
                   triple_plays INTEGER,
                   PRIMARY KEY (team_id, game_id),
                   FOREIGN KEY (team_id) REFERENCES team(team_id),
                   FOREIGN KEY (game_id) REFERENCES game(game_id),
                   FOREIGN KEY (team_id) REFERENCES team(team_id));"""

team_fill = """INSERT INTO team_appearance
               SELECT
                   h_name,
                   game_id,
                   1 AS home,
                   h_league,
                   h_score,
                   h_line_score,
                   h_at_bats,
                   h_hits,
                   h_doubles,
                   h_triples,
                   h_homeruns,
                   h_rbi,
                   h_sacrifice_hits,
                   h_sacrifice_flies,
                   h_hit_by_pitch,
                   h_walks,
                   h_intentional_walks,
                   h_strikeouts,
                   h_stolen_bases,
                   h_caught_stealing,
                   h_grounded_into_double,
                   h_first_catcher_interference,
                   h_left_on_base,
                   h_pitchers_used,
                   h_individual_earned_runs,
                   h_team_earned_runs,
                   h_wild_pitches,
                   h_balks,
                   h_putouts,
                   h_assists,
                   h_errors,
                   h_passed_balls,
                   h_double_plays,
                   h_triple_plays
               FROM game_log

               UNION

               SELECT    
                   v_name,
                   game_id,
                   0 AS home,
                   v_league,
                   v_score,
                   v_line_score,
                   v_at_bats,
                   v_hits,
                   v_doubles,
                   v_triples,
                   v_homeruns,
                   v_rbi,
                   v_sacrifice_hits,
                   v_sacrifice_flies,
                   v_hit_by_pitch,
                   v_walks,
                   v_intentional_walks,
                   v_strikeouts,
                   v_stolen_bases,
                   v_caught_stealing,
                   v_grounded_into_double,
                   v_first_catcher_interference,
                   v_left_on_base,
                   v_pitchers_used,
                   v_individual_earned_runs,
                   v_team_earned_runs,
                   v_wild_pitches,
                   v_balks,
                   v_putouts,
                   v_assists,
                   v_errors,
                   v_passed_balls,
                   v_double_plays,
                   v_triple_plays
               FROM game_log;"""
try:
    run_command(team_exec)
    run_command(team_fill)
except Exception:
    pass
team_check = """SELECT * FROM team_appearance LIMIT 5;"""
print(run_query(team_check))

In [ ]:
pa_exec = """CREATE TABLE person_appearance (
    appearance_id INTEGER PRIMARY KEY,
    person_id TEXT,
    team_id TEXT,
    game_id TEXT,
    appearance_type_id,
    FOREIGN KEY (person_id) REFERENCES person(person_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id));"""

pa_fill = """INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        NULL,
        hp_umpire_id,
        "UHP"
    FROM game_log
    WHERE hp_umpire_id IS NOT NULL    

UNION

    SELECT
        game_id,
        NULL,
        [1b_umpire_id],
        "U1B"
    FROM game_log
    WHERE "1b_umpire_id" IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [2b_umpire_id],
        "U2B"
    FROM game_log
    WHERE [2b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [3b_umpire_id],
        "U3B"
    FROM game_log
    WHERE [3b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        lf_umpire_id,
        "ULF"
    FROM game_log
    WHERE lf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        rf_umpire_id,
        "URF"
    FROM game_log
    WHERE rf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_manager_id,
        "MM"
    FROM game_log
    WHERE v_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_manager_id,
        "MM"
    FROM game_log
    WHERE h_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_pitcher_id,
        "AWP"
    FROM game_log
    WHERE winning_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score < v_score THEN h_name
            ELSE v_name
            END,
        losing_pitcher_id,
        "ALP"
    FROM game_log
    WHERE losing_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        saving_pitcher_id,
        "ASP"
    FROM game_log
    WHERE saving_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_rbi_batter_id,
        "AWB"
    FROM game_log
    WHERE winning_rbi_batter_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE v_starting_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE h_starting_pitcher_id IS NOT NULL;"""

pa_template = """INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "O{num}"
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL

UNION

    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "D" || CAST({hv}_player_{num}_def_pos AS INT)
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL;"""

try:
    run_command(pa_exec)
    run_command(pa_fill)

    for hv in ["h","v"]:
        for num in range(1,10):
            query_vars = {"hv": hv, "num": num}
            run_command(pa_template.format(**query_vars))
except Exception:
    pass
pa_check = """SELECT * FROM person_appearance LIMIT 5;"""
print(run_query(pa_check))

In [ ]:
try:
    run_command('DROP TABLE game_log;')
    run_command('DROP TABLE park_codes;')
    run_command('DROP TABLE person_codes;')
    run_command('DROP TABLE team_codes;')
    run_command('DROP TABLE appearance_type_table;')
except Exception:
    pass